### Code to calculate the partial correlation of temperature with ENSO, IOD and SAM indices 

This code used data from the moored location'NRSKAI', calculates the monthly temperature anomalies 

In [101]:
import sys
import pandas as pd
sys.path.insert(0, "../sa_upwelling")   # Allows using the files in the Python package
import utils

from matplotlib import pyplot as plt
#import pingouin as pg
from scipy.stats import pearsonr

In [4]:
hourly_files, hourly_datasets = utils.load_data_products()

Loading local hourly-timeseries data for mooring 'NRSKAI'.
Loading local hourly-timeseries data for mooring 'SAM8SG'.
Loading local hourly-timeseries data for mooring 'SAM5CB'.
Loading local hourly-timeseries data for mooring 'SAM2CP'.
Loading local hourly-timeseries data for mooring 'SAM6IS'.
Loading local hourly-timeseries data for mooring 'SAM3MS'.
Loading local hourly-timeseries data for mooring 'SAM7DS'.


In [6]:
# Run the extraction function for all sites, saving to CSV and storing the
# timeseries DataFrames in a local variable for subsequent use

temp_timeseries = dict()
for mooring in hourly_datasets.keys():
    ds = hourly_datasets[mooring]
    df = utils.extract_timeseries_df(ds, save=True)
    temp_timeseries[mooring] = df

,TIME,TEMP,DEPTH
0,2011-02-22 08:00:00,8.831907,517.403564
1,2011-02-22 09:00:00,8.842842,517.266357
2,2011-02-22 10:00:00,8.882489,517.178223
3,2011-02-22 11:00:00,8.976489,517.139893
4,2011-02-22 12:00:00,8.978115,517.163025
...,...,...,...
8183,2013-11-02 22:00:00,8.960722,512.247314
8184,2013-11-02 23:00:00,8.870369,512.339172
8185,2013-11-03 00:00:00,8.848995,512.442993
8186,2013-11-03 01:00:00,8.900201,512.533875


In [86]:
mor = pd.read_csv('../Datasets/NRSKAI_TEMP_101-111m.csv')
mor['TIME'] = pd.to_datetime(mor['TIME'])
mor.index = mor['TIME']
mor_mon = mor.resample('M').mean()
mor_mon = mor_mon.drop(['DEPTH'],axis=1)
mor_mon

/tmp/ipykernel_3898/1053529505.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mor_mon = mor.resample('M').mean()


,TEMP
TIME,
2008-08-31,14.256375
2008-09-30,14.363663
2008-10-31,14.873148
2008-11-30,15.532777
2008-12-31,16.153825
...,...
2023-01-31,14.030186
2023-02-28,14.010088
2023-03-31,15.036819


In [93]:
# Calculate the mean for each month across all years
monthly_means = mor_mon.groupby(mor_mon.index.month).mean()

# Calculate the anomalies by subtracting the monthly means from the data
anomalies = mor_mon - monthly_means.iloc[mor_mon.index.month - 1].values
anomalies

,TEMP
TIME,
2008-08-31,-0.515712
2008-09-30,-0.453968
2008-10-31,-0.145784
2008-11-30,0.424062
2008-12-31,1.009310
...,...
2023-01-31,-0.284630
2023-02-28,0.718094
2023-03-31,1.430747


In [38]:
soi = pd.read_csv('../Datasets/SOI_index.csv')
soi['date'] = pd.to_datetime(soi['date'])
soi.set_index('date',inplace=True)

/tmp/ipykernel_3898/2358500455.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  soi['date'] = pd.to_datetime(soi['date'])


,soi_index
date,
1990-01-31,-0.1
1990-02-28,-3.0
1990-03-31,-0.7
1990-04-30,0.3
1990-05-31,2.0
...,...
2020-08-31,1.8
2020-09-30,1.5
2020-10-31,0.8


In [48]:
iod = pd.read_csv('../Datasets/iod_index.csv')
iod['date'] = pd.to_datetime(iod['date'])
iod.set_index('date',inplace=True)

sam = pd.read_csv('../Datasets/SAM_index.csv')
sam['date'] = pd.to_datetime(sam['date'])
sam.set_index('date',inplace=True)

sam

/tmp/ipykernel_3898/2385061590.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  iod['date'] = pd.to_datetime(iod['date'])
/tmp/ipykernel_3898/2385061590.py:6: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  sam['date'] = pd.to_datetime(sam['date'])


,sam_index
date,
1990-01-31,-0.23
1990-02-28,1.95
1990-03-31,0.80
1990-04-30,-3.34
1990-05-31,-3.35
...,...
2020-08-31,-2.20
2020-09-30,-0.25
2020-10-31,1.79


In [103]:
start = max(anomalies.index.min(),soi.index.min())
end = min(anomalies.index.max(),soi.index.max())
anomalies_new = anomalies.loc[start:end]
soi_new = soi.loc[start:end]
iod_new = iod.loc[start:end]
sam_new = sam.loc[start:end]

new_df = pd.concat([anomalies_new,soi_new,iod_new,sam_new],axis=1)
#new_df.to_csv('trial.csv')

In [117]:

# Assuming new_df is your pandas DataFrame with columns: soi_index, iod_index, sam_index, TEMP

# Remove rows with NaN values
df_cleaned = new_df.dropna()

# Calculate the correlation matrix
correlation_matrix = df_cleaned[['soi_index', 'iod_index', 'sam_index', 'TEMP']].corr()

# Calculate the inverse of the correlation matrix
inv_corr_matrix = np.linalg.inv(correlation_matrix.values)

# Calculate partial correlations
partial_corr_enso = -inv_corr_matrix[0, 3] / np.sqrt(inv_corr_matrix[0, 0] * inv_corr_matrix[3, 3])
partial_corr_sam = -inv_corr_matrix[2, 3] / np.sqrt(inv_corr_matrix[2, 2] * inv_corr_matrix[3, 3])
partial_corr_iod = -inv_corr_matrix[1, 3] / np.sqrt(inv_corr_matrix[1, 1] * inv_corr_matrix[3, 3])

In [115]:
correlation_matrix

,soi_index,iod_index,sam_index,TEMP
soi_index,1.000000,-0.289515,0.132069,0.157159
iod_index,-0.289515,1.000000,-0.096046,-0.099271
sam_index,0.132069,-0.096046,1.000000,-0.162073
TEMP,0.157159,-0.099271,-0.162073,1.000000


In [116]:
# Print the results
print("Partial correlation between ENSO and TEMP:", partial_corr_enso)
print("Partial correlation between SAM and TEMP:", partial_corr_sam)
print("Partial correlation between IOD and TEMP:", partial_corr_iod)

Partial correlation between ENSO and TEMP: 0.15709550042969012
Partial correlation between SAM and TEMP: -0.19089438188243144
Partial correlation between IOD and TEMP: -0.06961572415626517
